In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

Mounted at /content/drive
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project/spectroconv
     |████████████████████████████████| 348 kB 4.2 MB/s 
     |████████████████████████████████| 81 kB 13.0 MB/s 
     |████████████████████████████████| 132 kB 61.6 MB/s 
     |████████████████████████████████| 6.2 MB 69.7 MB/s 
     |████████████████████████████████| 79 kB 4.8 MB/s 
     |████████████████████████████████| 8.7 MB 49.9 MB/s 
     |████████████████████████████████| 138 kB 76.6 MB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 29 07:33:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from models import wb_experiment

NumExpr defaulting to 2 threads.


In [33]:
n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': 'cnn' # use this only when training/evaluating/etc a CNN
}

config = {
    'data': {
        'train': {
            'name': f'nsynth-full-{n_mels}-train'
        },
        'val': {
            'name': f'nsynth-full-{n_mels}-val'
        }
    },
    'model': {
        'id': wb_experiment.ModelType.CNN,
        'name': 'CNN'
    },
    'logging': {
        'batch_log_interval': 250,
        'eval_every_batch': True,
        'eval_every_epoch': True,
    },
    'parameters': {
      'final_filter_size': 16,
      'num_conv_layers': 1,
      'linear_layer_size': 64,
      'optimizer': 'adam',
      'l2_reg': 0.001,
      'learning_rate': 0.0005,
      'dropout': 0.5, # probability of an element to be zeroed (0 = no dropout)
      'epochs': 3,
      'batch_size': 100,
      'kernel_size_start': 6,
    },
}

In [ ]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=True)
trainer.run_train(config, save_model=False)

In [ ]:
# untested

config = {
    'data': {
        'val': {
            'name': f'nsynth-full-{n_mels}-val'
        }
    },
    'model_name': 'CNN',
    'eval': {
        'examples': {
            'k': 8
        }
    }
}


In [ ]:

trainer = wb_experiment.EvalExperiment(wb_config)
metrics, examples_df = trainer.run_evaluate(config)

## Train on the full dataset

In [ ]:
# Wandb should take care of plotting for us?

'''
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(losses)
fig, ax = plt.subplots()
ax.plot(train_accuracies, label='train acc')
ax.plot(val_accuracies, label='val acc')
ax.plot(val_f1s, label='val f1')
ax.legend()
plt.savefig("learning_metrics.png")
plt.show()
'''

In [ ]:
#print(train_model.stream_accuracy(clf, dl_val, device))

0.7415788789179347


In [ ]:
#print(train_model.stream_f1(clf, dl_val, device))

0.7109029348873568
